In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import math
import seaborn as sns

import sklearn
from sklearn.preprocessing import OneHotEncoder

In [3]:
sklearn.__version__

'0.21.2'

## LendingClub Data Acquisition

In [4]:
# raw_q12019 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2019Q1.csv',header=1)
# raw_q42018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q4.csv',header=1)
# raw_q32018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q3.csv',header=1)
# raw_q22018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q2.csv',header=1)
# raw_q12018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q1.csv',header=1)
raw_q42017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q4.csv',header=1)
raw_q32017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q3.csv',header=1)
raw_q22017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q2.csv',header=1)
raw_q12017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q1.csv',header=1)
raw_q42016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q4.csv',header=1)
raw_q32016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q3.csv',header=1)
raw_q22016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q2.csv',header=1)
raw_q12016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q1.csv',header=1)
raw_2015 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2015.csv',header=1)
raw_2014 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2014.csv',header=1)
raw_2012to2013 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2012to2013.csv',header=1)
raw_2007to2011 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2007to2011.csv',header=1)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,118) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,19,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/

In [5]:
# print(raw_q12019.shape)
# print(raw_q42018.shape)
# print(raw_q32018.shape)
# print(raw_q22018.shape)
# print(raw_q12018.shape)
print(raw_q42017.shape)
print(raw_q32017.shape)
print(raw_q22017.shape)
print(raw_q12017.shape)
print(raw_q42016.shape)
print(raw_q32016.shape)
print(raw_q22016.shape)
print(raw_q12016.shape)
print(raw_2015.shape)
print(raw_2014.shape)
print(raw_2012to2013.shape)
print(raw_2007to2011.shape)
#raw_q12019,raw_q42018,raw_q32018,raw_q22018,raw_q12018,
dfs = [raw_q42017,raw_q32017,raw_q22017,raw_q12017,
       raw_q42016,raw_q32016,raw_q22016,raw_q12016,raw_2015,
       raw_2014,raw_2012to2013, raw_2007to2011]

(118650, 150)
(122703, 150)
(105453, 150)
(96781, 150)
(103548, 150)
(99122, 150)
(97856, 150)
(133889, 150)
(421097, 150)
(235631, 150)
(188183, 150)
(42538, 150)


In [6]:
raw_lc_df = pd.concat(dfs, ignore_index=True)

In [7]:
raw_lc_df['loan_status'].value_counts()

Fully Paid                                             1076208
Current                                                 399465
Charged Off                                             268335
Late (31-120 days)                                       12138
In Grace Period                                           3934
Late (16-30 days)                                         2586
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     11
Name: loan_status, dtype: int64

In [8]:
raw_lc_df.shape

(1765451, 150)

## Data Cleaning

In [9]:
#FILTER OUT WHOSE STATUS IS NOT YET FINALIZED
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'Current']
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'Late (31-120 days)']
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'Late (16-30 days)']
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'In Grace Period']
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'Does not meet the credit policy. Status:Fully Paid']
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'Does not meet the credit policy. Status:Charged Off']

#BINARIZE CLASSES
raw_lc_df['loan_status'] = raw_lc_df['loan_status'].map({'Charged Off': 0, 'Default': 0, 'Fully Paid': 1})

In [10]:
raw_lc_df['earliest_cr_line'] = pd.to_timedelta(pd.to_datetime(raw_lc_df['earliest_cr_line'])).dt.days

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  """Entry point for launching an IPython kernel.


In [11]:
def parse_percentage(percentage):
    '''Change percentage features into floats'''
    if str(percentage) == 'nan': return math.nan
    new = percentage.replace('%', '')
    return float(new) / 100.0

raw_lc_df['revol_util'] = raw_lc_df['revol_util'].apply(parse_percentage)
raw_lc_df['int_rate'] = raw_lc_df['int_rate'].apply(parse_percentage)

In [12]:
#FILTER OUT COLUMNS THAT ARE OBVIOUSLY REDUNDANT OR UNHELPFUL
lc_df = raw_lc_df[['loan_amnt', 'funded_amnt',
                'term', 'int_rate', 'installment',
                'emp_length', 'home_ownership', 'annual_inc',
                'verification_status', 'loan_status',
                'purpose', 'zip_code', 'addr_state', 'dti',
                'delinq_2yrs', 'earliest_cr_line', 'fico_range_low',
                'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq',
                'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
                'revol_util', 'total_acc', 'out_prncp',
                'out_prncp_inv', 'last_fico_range_high', 'last_fico_range_low',
                'collections_12_mths_ex_med', 'mths_since_last_major_derog',
                'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
                'acc_now_delinq', 'tot_coll_amt',
                'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
                'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
                'open_rv_12m', 'open_rv_24m', 'all_util',
                'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
                'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
                'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
                'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
                'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
                'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
                'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
                'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
                'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
                'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m',
                'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75',
                'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim',
                'total_bal_ex_mort', 'total_bc_limit',
                'total_il_high_credit_limit','grade', 'collection_recovery_fee', 
                'total_rec_prncp', 'title', 'total_rec_int', 'total_rec_late_fee', 
                'sub_grade', 'debt_settlement_flag', 'emp_title','issue_d','last_pymnt_d']]

In [13]:
lc_df.shape

(1344579, 101)

In [14]:
lc_df = lc_df.dropna(axis=0, subset=['loan_amnt','inq_last_6mths']).reset_index(drop=True)

In [15]:
#CAST DATATYPES
dtype = {
      'loan_amnt': 'int64',
      'term': 'object',
      'int_rate': 'float64',
      'emp_length': 'object',
      'home_ownership': 'object',
      'annual_inc': 'float64',
      'verification_status': 'object',
      'loan_status': 'object',
      'purpose': 'object',
      'zip_code': 'object',
      'addr_state': 'object',
      'dti': 'float64',
      'delinq_2yrs': 'int64',
      'fico_range_low': 'int64',
      'fico_range_high': 'int64',
      'inq_last_6mths': 'int64',
      'open_acc': 'int64',
      'pub_rec': 'int64',
      'revol_bal': 'int64',
      'total_acc': 'int64',
      'application_type': 'object'
    }
lc_df = lc_df.astype(dtype=dtype)

In [16]:
lc_df.head().T

,0,1,2,3,4
loan_amnt,30000,13000,15000,15750,19800
funded_amnt,30000,13000,15000,15750,19800
term,60 months,60 months,36 months,36 months,60 months
int_rate,0.3079,0.1505,0.1042,0.0735,0.1505
installment,985.21,309.62,486.98,488.84,471.57
emp_length,10+ years,10+ years,3 years,10+ years,3 years
home_ownership,RENT,RENT,RENT,RENT,RENT
annual_inc,40000,72000,55000,96500,108600
verification_status,Source Verified,Source Verified,Source Verified,Source Verified,Not Verified
loan_status,0,1,1,1,1


In [17]:
lc_df.columns[lc_df.isna().all()].tolist()

[]

In [18]:
lc_df.shape

(1344553, 101)

In [19]:
lc_df[['addr_state','annual_inc','collection_recovery_fee','emp_title',
           'fico_range_high','fico_range_low','funded_amnt','grade','home_ownership',
           'int_rate','loan_amnt','loan_status','purpose','sub_grade','title','total_rec_int',
           'total_rec_late_fee','total_rec_prncp','zip_code','debt_settlement_flag','out_prncp',
           'collection_recovery_fee']].head(20)
   

,addr_state,annual_inc,collection_recovery_fee,emp_title,fico_range_high,fico_range_low,funded_amnt,grade,home_ownership,int_rate,...,purpose,sub_grade,title,total_rec_int,total_rec_late_fee,total_rec_prncp,zip_code,debt_settlement_flag,out_prncp,collection_recovery_fee
0,NY,40000.0,0.0000,Supervsir,704,700,30000.0,G,RENT,0.3079,...,other,G1,Other,9463.05,98.52,2196.71,103xx,N,0.0,0.0000
1,FL,72000.0,0.0000,Sales manager,709,705,13000.0,C,RENT,0.1505,...,debt_consolidation,C4,Debt consolidation,598.13,0.00,13000.00,333xx,N,0.0,0.0000
2,WA,55000.0,0.0000,Finance,674,670,15000.0,B,RENT,0.1042,...,debt_consolidation,B3,Debt consolidation,1065.25,0.00,15000.00,981xx,N,0.0,0.0000
3,HI,96500.0,0.0000,Service Coordinator,709,705,15750.0,A,RENT,0.0735,...,debt_consolidation,A4,Debt consolidation,641.95,0.00,15750.00,968xx,N,0.0,0.0000
4,MA,108600.0,0.0000,engineer,729,725,19800.0,C,RENT,0.1505,...,credit_card,C4,Credit card refinancing,2471.93,0.00,19800.00,021xx,N,0.0,0.0000
5,NJ,100000.0,0.0000,Paralegal,779,775,10000.0,A,MORTGAGE,0.0532,...,debt_consolidation,A1,Debt consolidation,78.81,0.00,10000.00,087xx,N,0.0,0.0000
6,CA,55000.0,0.0000,Admissions Advisor,664,660,32600.0,B,RENT,0.1199,...,house,B5,Home buying,4036.10,0.00,32600.00,958xx,N,0.0,0.0000
7,NJ,54000.0,0.0000,Electro Mechanical Technician,669,665,10000.0,B,RENT,0.0944,...,debt_consolidation,B1,Debt consolidation,696.77,0.00,10000.00,070xx,N,0.0,0.0000
8,IN,85000.0,0.0000,Special Education Teacher,664,660,13000.0,B,MORTGAGE,0.0993,...,debt_consolidation,B2,Debt consolidation,569.54,0.00,13000.00,462xx,N,0.0,0.0000
9,FL,70000.0,200.5740,RN,694,690,10000.0,A,MORTGAGE,0.0672,...,other,A3,Other,371.08,46.14,1788.05,327xx,N,0.0,200.5740


### Feature Preprocessing

#### Mean, Max, Zero Impute for Missing Values

In [20]:
#replace NaNs with column max, column means, or zeros where it makes sense to
nan_max_cols = [
    'mths_since_last_delinq',
    'mths_since_last_record',
    'mths_since_last_major_derog',
    'mths_since_rcnt_il',
    'mths_since_recent_bc',
    'mths_since_recent_bc_dlq',
    'mths_since_recent_inq',
    'mths_since_recent_revol_delinq',
    'mo_sin_old_il_acct',
    'mo_sin_old_rev_tl_op',
    'mo_sin_rcnt_rev_tl_op',
    'mo_sin_rcnt_tl',
]

nan_zero_cols = [
    'tot_coll_amt',
    'tot_cur_bal',
    'open_acc_6m',
    'open_act_il',
    'open_il_12m',
    'open_il_24m',
    'total_bal_il',
    'open_rv_12m',
    'open_rv_24m',
    'inq_fi',
    'inq_last_12m',
    'num_accts_ever_120_pd',
    'num_actv_bc_tl',
    'num_actv_rev_tl',
    'num_bc_sats',
    'num_bc_tl',
    'num_il_tl',
    'num_op_rev_tl',
    'num_rev_accts',
    'num_rev_tl_bal_gt_0',
    'num_sats',
    'num_tl_120dpd_2m',
    'num_tl_30dpd',
    'num_tl_90g_dpd_24m',
    'num_tl_op_past_12m',
    'mort_acc',
    'percent_bc_gt_75',
    'tot_hi_cred_lim',
    'total_bal_ex_mort',
    'total_bc_limit',
    'total_il_high_credit_limit',
    'bc_open_to_buy',
    'acc_open_past_24mths',
    'total_cu_tl',
    'total_rev_hi_lim',
]

nan_mean_cols = [
    'dti',
    'annual_inc_joint',
    'dti_joint',
    'il_util',
    'all_util',
    'bc_util',
    'pct_tl_nvr_dlq',
    'avg_cur_bal',
    'revol_util',
]

for col in lc_df:
    dt = lc_df[col].dtype
    #FILL MISSING WITH MEAN
    if col in nan_mean_cols:
        lc_df[col] = lc_df[col].fillna(np.nanmean(lc_df[col].values))
    #FILL MISSING WITH ZEROS
    if col in nan_zero_cols:
        lc_df[col] = lc_df[col].fillna(0.0)
    #FILL MISSING WITH MAX
    if col in nan_max_cols:
        lc_df[col] = lc_df[col].fillna(np.nanmax(lc_df[col].values) * 5)

In [21]:
#Change dtype
lc_df.loc[lc_df['emp_length'] == '< 1 year','emp_length'] = '0'
lc_df.loc[lc_df['emp_length'] == '10+ years', 'emp_length'] = '10'
lc_df['emp_length'] = lc_df['emp_length'].str[:1]
lc_df['emp_length'] = lc_df['emp_length'].astype(float)

In [22]:
lc_df.info(verbose=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344553 entries, 0 to 1344552
Data columns (total 101 columns):
loan_amnt                         int64
funded_amnt                       float64
term                              object
int_rate                          float64
installment                       float64
emp_length                        float64
home_ownership                    object
annual_inc                        float64
verification_status               object
loan_status                       object
purpose                           object
zip_code                          object
addr_state                        object
dti                               float64
delinq_2yrs                       int64
earliest_cr_line                  float64
fico_range_low                    int64
fico_range_high                   int64
inq_last_6mths                    int64
mths_since_last_delinq            float64
mths_since_last_record            float64
open_acc             

In [23]:
lc_df.shape

(1344553, 101)

In [24]:
clean_lc_df = lc_df.copy()

### Categorical Variable One-Hot Encoding

In [25]:
#Drop nulls before OHE
clean_lc_df = clean_lc_df[clean_lc_df['zip_code'].notnull()].reset_index(drop=True)
clean_lc_df = clean_lc_df[clean_lc_df['emp_title'].notnull()].reset_index(drop=True)

In [26]:
clean_lc_df.shape

(1260156, 101)

###### One-Hot Encode "term"

In [27]:
clean_lc_df['term'].value_counts()

 36 months    949716
 60 months    310440
Name: term, dtype: int64

In [28]:
#term
encoder = OneHotEncoder(drop='first',categories='auto').fit(clean_lc_df[['term']])
encoder.categories_

[array([' 36 months', ' 60 months'], dtype=object)]

In [29]:
ohe_term = pd.DataFrame(encoder.transform(clean_lc_df[['term']]).toarray(),
                   columns=encoder.get_feature_names(["term"]))

ohe_term.head()

,term_ 60 months
0,1.0
1,1.0
2,0.0
3,0.0
4,1.0


###### One-Hot Encode "home_ownership"

In [30]:
clean_lc_df['home_ownership'].value_counts()

MORTGAGE    624692
RENT        507265
OWN         127760
ANY            257
OTHER          138
NONE            44
Name: home_ownership, dtype: int64

In [31]:
encoder = OneHotEncoder(drop='first',categories='auto').fit(clean_lc_df[['home_ownership']])
ohe_home_ownership = pd.DataFrame(encoder.transform(clean_lc_df[['home_ownership']]).toarray(),
                   columns=encoder.get_feature_names(["home_ownership"]))

ohe_home_ownership.head()

,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0


##### One-Hot Encode "purpose"

In [32]:
clean_lc_df['purpose'].value_counts()

debt_consolidation    737354
credit_card           277142
home_improvement       80184
other                  70405
major_purchase         27107
small_business         14073
medical                13888
car                    13514
moving                  8740
vacation                8264
house                   6106
wedding                 2219
renewable_energy         859
educational              301
Name: purpose, dtype: int64

In [33]:
encoder = OneHotEncoder(drop='first',categories='auto').fit(clean_lc_df[['purpose']])
ohe_purpose = pd.DataFrame(encoder.transform(clean_lc_df[['purpose']]).toarray(),
                   columns=encoder.get_feature_names(["purpose"]))

ohe_purpose.head()

,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### One-Hot Encode "zip_code"

In [34]:
clean_lc_df['zip_code'].value_counts() #945 ZIP CODES

945xx    14063
750xx    13968
112xx    13146
606xx    11898
300xx    11478
331xx    10900
070xx    10398
100xx    10174
900xx    10107
770xx    10082
891xx     9539
330xx     9485
917xx     9105
104xx     8851
117xx     8770
852xx     8092
921xx     7901
926xx     7638
913xx     7496
925xx     7302
334xx     6909
113xx     6633
481xx     6616
601xx     6541
604xx     6417
853xx     6378
021xx     6367
760xx     6283
600xx     6249
080xx     6225
         ...  
511xx        1
269xx        1
525xx        1
568xx        1
849xx        1
095xx        1
819xx        1
892xx        1
698xx        1
524xx        1
516xx        1
817xx        1
889xx        1
552xx        1
429xx        1
521xx        1
694xx        1
507xx        1
862xx        1
896xx        1
872xx        1
888xx        1
202xx        1
771xx        1
520xx        1
964xx        1
649xx        1
643xx        1
509xx        1
938xx        1
Name: zip_code, Length: 944, dtype: int64

In [35]:
encoder = OneHotEncoder(drop='first',categories='auto').fit(clean_lc_df[['zip_code']])
ohe_zip_code = pd.DataFrame(encoder.transform(clean_lc_df[['zip_code']]).toarray(),
                   columns=encoder.get_feature_names(["zip_code"]))

ohe_zip_code.head()

,zip_code_008xx,zip_code_009xx,zip_code_010xx,zip_code_011xx,zip_code_012xx,zip_code_013xx,zip_code_014xx,zip_code_015xx,zip_code_016xx,zip_code_017xx,...,zip_code_990xx,zip_code_991xx,zip_code_992xx,zip_code_993xx,zip_code_994xx,zip_code_995xx,zip_code_996xx,zip_code_997xx,zip_code_998xx,zip_code_999xx
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### One-Hot Encode "addr_state"

In [36]:
clean_lc_df['addr_state'].value_counts()

CA    183996
TX    104695
NY    103610
FL     87456
IL     49364
NJ     46287
PA     42936
OH     41112
GA     40642
VA     36118
NC     34856
MI     32051
AZ     29834
MA     29597
MD     29563
CO     27749
WA     27191
MN     22912
IN     20441
MO     19944
TN     18995
CT     18863
NV     18100
WI     16688
AL     15343
OR     15070
LA     14695
SC     14591
KY     11928
OK     11408
KS     10638
UT      9319
AR      9172
NM      6740
HI      6441
MS      6128
NH      6126
RI      5581
WV      4327
MT      3556
DE      3490
NE      3385
DC      3325
AK      3068
WY      2802
SD      2640
VT      2499
ME      1872
ND      1564
ID      1441
IA         7
Name: addr_state, dtype: int64

In [37]:
encoder = OneHotEncoder(drop='first',categories='auto').fit(clean_lc_df[['addr_state']])
ohe_addr_state = pd.DataFrame(encoder.transform(clean_lc_df[['addr_state']]).toarray(),
                   columns=encoder.get_feature_names(["addr_state"]))

ohe_addr_state.head()

,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### One-Hot Encode "application_type"

In [38]:
clean_lc_df['application_type'].value_counts()

Individual    1242027
Joint App       18129
Name: application_type, dtype: int64

In [39]:
encoder = OneHotEncoder(drop='first',categories='auto').fit(clean_lc_df[['application_type']])
ohe_application_type = pd.DataFrame(encoder.transform(clean_lc_df[['application_type']]).toarray(),
                   columns=encoder.get_feature_names(["application_type"]))

ohe_application_type.head()

,application_type_Joint App
0,1.0
1,0.0
2,0.0
3,0.0
4,1.0


##### One-Hot Encode "grade"

In [40]:
clean_lc_df['grade'].value_counts()

B    369420
C    359150
A    219745
D    184916
E     87784
F     30363
G      8778
Name: grade, dtype: int64

In [41]:
encoder = OneHotEncoder(drop='first',categories='auto').fit(clean_lc_df[['grade']])
ohe_grade = pd.DataFrame(encoder.transform(clean_lc_df[['grade']]).toarray(),
                   columns=encoder.get_feature_names(["grade"]))

ohe_grade.head()

,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G
0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0


##### One-Hot Encode "sub_grade"

In [42]:
clean_lc_df['sub_grade'].value_counts()

C1    80676
B4    78150
B5    77667
B3    77432
C2    74634
C3    70448
C4    69905
B2    69330
B1    66841
C5    63487
A5    60100
A4    48257
D1    47538
A1    42372
D2    41352
D3    36158
A3    34755
A2    34261
D4    32646
D5    27222
E1    22457
E2    20261
E3    17222
E4    14665
E5    13179
F1     9287
F2     6860
F3     5797
F4     4660
F5     3759
G1     2798
G2     2076
G3     1552
G4     1265
G5     1087
Name: sub_grade, dtype: int64

In [43]:
encoder = OneHotEncoder(drop='first',categories='auto').fit(clean_lc_df[['sub_grade']])
ohe_sub_grade = pd.DataFrame(encoder.transform(clean_lc_df[['sub_grade']]).toarray(),
                   columns=encoder.get_feature_names(["sub_grade"]))

ohe_sub_grade.head()

,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,...,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### One-Hot Encode "emp_title"

In [44]:
counts = clean_lc_df['emp_title'].value_counts()
idx = counts[counts.lt(1600)].index

clean_lc_df.loc[clean_lc_df['emp_title'].isin(idx) == False, 'emp_title_2'] = clean_lc_df['emp_title']
clean_lc_df.loc[clean_lc_df['emp_title'].isin(idx), 'emp_title_2'] = 'Other'

In [45]:
len(clean_lc_df['emp_title_2'].value_counts())

50

In [46]:
encoder = OneHotEncoder(drop='first',categories='auto').fit(clean_lc_df[['emp_title_2']])
ohe_emp_title_2 = pd.DataFrame(encoder.transform(clean_lc_df[['emp_title_2']]).toarray(),
                   columns=encoder.get_feature_names(["emp_title_2"]))

ohe_emp_title_2.head()

,emp_title_2_Account Manager,emp_title_2_Accountant,emp_title_2_Administrative Assistant,emp_title_2_Administrator,emp_title_2_Analyst,emp_title_2_Assistant Manager,emp_title_2_Attorney,emp_title_2_Branch Manager,emp_title_2_CEO,emp_title_2_Consultant,...,emp_title_2_Technician,emp_title_2_Truck Driver,emp_title_2_Vice President,emp_title_2_driver,emp_title_2_manager,emp_title_2_owner,emp_title_2_sales,emp_title_2_supervisor,emp_title_2_teacher,emp_title_2_truck driver
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Join lc_df with new One-Hot-Encoded DataFrames
- lc_df + ohe_home_ownership + ohe_purpose + ohe_zip_code + ohe_application_type + ohe_sub_grade(more granular than ohe_grade) + ohe_emp_title_2
- I'm opting for Zip Code (944 cols) instead of State (50 cols - D.C. counts) in the first model under the hypothesis that the more granular location-based variable (zip) will reveal a more meaningful relationship than the broader borders of State

In [47]:
#Individual DF shapes
print(clean_lc_df.shape)
print(ohe_home_ownership.shape)
print(ohe_purpose.shape)
print(ohe_zip_code.shape)
print(ohe_addr_state.shape)
print(ohe_sub_grade.shape)
print(ohe_emp_title_2.shape)

(1260156, 102)
(1260156, 5)
(1260156, 13)
(1260156, 943)
(1260156, 50)
(1260156, 34)
(1260156, 49)


In [48]:
#Future Shape
print(clean_lc_df.shape[1]+ohe_home_ownership.shape[1]+ohe_purpose.shape[1]+
      ohe_zip_code.shape[1]+ohe_sub_grade.shape[1]+
     ohe_emp_title_2.shape[1])

1146


In [49]:
concat_dfs = [clean_lc_df, ohe_home_ownership, ohe_purpose, ohe_zip_code,
              ohe_application_type, ohe_sub_grade, ohe_emp_title_2]
clean_df = pd.concat(concat_dfs,axis=1)

In [50]:
clean_df.to_csv('clean_df.csv')